### Задание 1
Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js

In [1]:
import requests

In [2]:
def name_currency_max_course():
    r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
    exchange_rates = r.json()['Valute']
    max_course = 0
    for currency_data in exchange_rates.values():
        if currency_data['Value'] > max_course:
            name_currency = currency_data['Name']
            max_course = currency_data['Value']
    return print(name_currency, max_course)

In [3]:
name_currency_max_course()

Датских крон 91.2613


### Задание 2

Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах eur и usd будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

In [4]:
# Полностью скопировал класс, ничего не меняя

class Rate:
    """
    
    """
    def __init__(self, format_='value'):
        self.format = format_
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return self.r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                return response[currency]['Value']
        
        return 'Error'
    
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR')
    
    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD')
    
    def brl(self):
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        return self.make_format('BRL')

In [6]:
# Создал класс дочерний от Rate

class Rate_diff(Rate):
    def __init__(self, diff=True, format_='value'):
        super().__init__(format_)
        self.diff = diff
    
    def make_format_diff(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        - информация только о разнице курса по 
        Rate('value').make_format('EUR')
        79.4966
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                if self.diff:
                    return response[currency]['Value'] - response[currency]['Previous']
                else:
                    return response[currency]['Value']
        
        return 'Error'
        
    def eur(self):
        """При format_='full' возвращает курс евро на сегодня в формате self.format
           При format_='value' возвращает разницу курса евро на сегодня по сравнению в прошлым значением"""
        return self.make_format_diff('EUR')
    
    def usd(self):
        """При format_='full' возвращает курс доллара на сегодня в формате self.format
           При format_='value' возвращает разницу курса доллара на сегодня по сравнению в прошлым значением"""
        return self.make_format_diff('USD')

In [7]:
Rate_diff().eur()

-0.3275999999999897

In [8]:
Rate_diff(format_='full').eur()

{'ID': 'R01239',
 'NumCode': '978',
 'CharCode': 'EUR',
 'Nominal': 1,
 'Name': 'Евро',
 'Value': 68.2082,
 'Previous': 68.5358}

In [9]:
Rate_diff(False).eur()

68.2082

In [10]:
Rate_diff(format_='value').make_format('EUR')

68.2082

### Задание 3

Напишите класс Designer, который учитывает количество международных премий для дизайнеров (из презентации: "Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла"). Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять). Выполните проверку для 20 аккредитаций дизайнера Елены.

Для проверки используйте код:

elena = Designer('Елена', seniority=0, awards=2)

for i in range(20):
    elena.check_if_it_is_time_for_upgrade()


In [11]:
class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority
        
        self.grade = 1
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)

In [12]:
class Designer(Employee):
    def __init__(self, name, seniority, awards=2):
        super().__init__(name, seniority)
        self.awards = awards
            
    def check_if_it_is_time_for_upgrade(self):
        # для каждой аккредитации увеличиваем счетчик на 1
        # пока считаем, что все разработчики проходят аккредитацию
        self.seniority += 1
        
        # условие повышения сотрудника из презентации
        if (self.seniority + 2*self.awards) % 7 == 0:
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()

In [13]:
elena = Designer('Елена', seniority=0, awards=2)

In [14]:
for i in range(20):
    elena.check_if_it_is_time_for_upgrade()

Елена 1
Елена 1
Елена 2
Елена 2
Елена 2
Елена 2
Елена 2
Елена 2
Елена 2
Елена 3
Елена 3
Елена 3
Елена 3
Елена 3
Елена 3
Елена 3
Елена 4
Елена 4
Елена 4
Елена 4
